In [2]:
import os
import requests
import shutil
import time
import sys
import re
import urllib
from bs4 import BeautifulSoup
import openpyxl
from openpyxl.cell.cell import ILLEGAL_CHARACTERS_RE
import MySQLdb

ModuleNotFoundError: No module named 'MySQLdb'

In [2]:
def tx_coding(text):
    # address UnicodeEncodeError: ‘cp950’ codec can’t encode character
    return text.encode(sys.stdin.encoding, "replace").decode(sys.stdin.encoding)

def get_web_page(url):
    resp = requests.get(
        url=url,
        cookies={'over18': '1'}
    )
  
    if resp.status_code != 200:
        return None
    else:
        return tx_coding(resp.text)
    
def get_next_page_url_and_parper_url(url,date):
    page = get_web_page(url)
    soup = BeautifulSoup(page, 'html.parser')
    #next_page_url
    d = soup.find('div', {'class': 'btn-group btn-group-paging'})
    next_page = "https://www.ptt.cc" + (d.find_all('a'))[1]["href"]
    #articles
    title = ""
    href = ""
    push_count = ""
    # 初始化一個【文章列表陣列】，儲存取得的文章資料
    articles = []  
    # 取得所有<div class:r-ent> block的資料，並放在divs陣列裡面
    divs = soup.find_all('div', {'class': 'r-ent'})
    for div in divs:
    # 如果發文日期是今天，且有超連結，表示文章存在，未被刪除
            ######################
            # 取得文章連結及標題			
        if (div.find('div', 'date').string == date) or (div.find('a')):
            push_count = div.find('div', {'class': 'nrec'}).string
            href = "https://www.ptt.cc" + div.find('a')['href']
            title = div.find('a').string
        else:
            continue
        
        # 忽略有[公告]兩字的文章 
        if title.find('公告') !=-1:
            continue
        pass
    
        ######################
        # 將取得的資料分類存在【文章列表陣列】
        articles.append({
            'local_url': href,
            'title': title,
            'push_count': push_count
        })
    print(url,"\n文章",len(articles))
    return (next_page,articles)

def get_paper_content(url):
    times=""
    page = get_web_page(url)
    soup = BeautifulSoup(page, 'html.parser')
    links = soup.find(id='main-content')
    links_text = soup.find(id='main-content').text
    
    for i in links.find_all(class_='push'):
        links_text = links_text.replace(str(i.text),"")
  
    for i in links.find_all(class_='f2'):
        links_text = links_text.replace(str(i.text),"")
  
    for i in links.find_all(class_='article-metaline'):
        times = i.find(class_='article-meta-value').text
        links_text = links_text.replace(str(i.text),"")
  
    for i in links.find_all(class_='article-metaline-right'):
        links_text = links_text.replace(str(i.text),"")
        links_text = links_text.replace(r'^https?://(i.)?(m.)?imgur.com','')
        links_text = ILLEGAL_CHARACTERS_RE.sub(r'', links_text)
    links_text = links_text.replace("\n","")
    return (links_text,times)

In [3]:
text,times="",""
first_page = "https://www.ptt.cc/bbs/prozac/index.html"
date = time.strftime("%m/%d").lstrip('0')
conn=MySQLdb.connect(host="127.0.0.1",user="root",passwd="1234",db="ptt",charset="utf8")
cursor=conn.cursor()
for page_count in range(10):
    next_page_url,articles = get_next_page_url_and_parper_url(first_page,date)
    for articles_count in range(len(articles)):
        text,times = get_paper_content(articles[articles_count]["local_url"])
        cursor.execute('INSERT INTO ptt_articles(title,content,time,push_count) VALUES(%s,%s,%s,%s)',(str(articles[articles_count]["title"]),str(text),str(times),str(articles[articles_count]["push_count"])))
        conn.commit()
        #time.sleep(2)
    first_page = next_page_url

https://www.ptt.cc/bbs/prozac/index.html 
文章 3
https://www.ptt.cc/bbs/prozac/index1226.html 
文章 17
https://www.ptt.cc/bbs/prozac/index1225.html 
文章 17
https://www.ptt.cc/bbs/prozac/index1224.html 
文章 16
https://www.ptt.cc/bbs/prozac/index1223.html 
文章 13
https://www.ptt.cc/bbs/prozac/index1222.html 
文章 16
https://www.ptt.cc/bbs/prozac/index1221.html 
文章 14
https://www.ptt.cc/bbs/prozac/index1220.html 
文章 16
https://www.ptt.cc/bbs/prozac/index1219.html 
文章 16
https://www.ptt.cc/bbs/prozac/index1218.html 
文章 18


In [91]:
date = time.strftime("%m/%d").lstrip('0')
next_page_url,articles = get_next_page_url_and_parper_url("https://www.ptt.cc/bbs/prozac/index.html",date)

In [92]:
next_page_url

'https://www.ptt.cc/bbs/prozac/index1222.html'

In [52]:
articles[0]

{'local_url': 'https://www.ptt.cc/bbs/prozac/M.1594228793.A.C39.html',
 'title': '[陰天] 陰晴不定',
 'push_count': '1'}

In [63]:
SQL="SELECT * FROM ptt_articles"
cursor.execute(SQL)
cursor.fetchall()[0][1]

"像是今天的天氣，以為是大晴天卻突然烏雲密佈，以為要下雨了終究是沒有。大概是因為我掙扎著想好起來，但是無法維持在好的狀態，瞬間消失的食慾竟然就這麼持續了幾天，可以工作，但無法高度專注。失去專注力這件事一直讓我很困擾，難道這是一個不可逆的過程嗎？連看個輕鬆的書都不斷走神QQ新聞今天依然在發酵，還是會忍不住被影響，關於男性與女性、強權與弱勢、傷害與被傷害、婚姻與家庭、愛與恨、生與死，不過也許一切都只是我自己腦補了太多，其實與我無關，太多的情緒感知太累了，沒有辦法承受的話還是不要想比較好。想著他就好了，今天他撐著想睡也要陪我說一會兒話，覺得暖暖的，很可愛。還好有他在，讓模糊的世界稍微清晰了一點。https://youtu.be/m0ffXI1kBvs今天是抒情感EDMYou're the best thing, you're a beautiful feeling I have--"

In [70]:
a= "kevin"
a.find('vin')

2